In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from lxml import etree
import re

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
    }

In [3]:
main_url = 'https://homecityaruba.com/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
    }
hyper = ['https://homecityaruba.com/for-sale',
         'https://homecityaruba.com/for-rent'
    ]

In [7]:
def get_all_url(li):
    all_links = []
    for url in li:
        req = requests.get(url)
        soup = BeautifulSoup(req.content, "html.parser")
        anchors = soup.find('div', {'class': 'preview-properties'})
        links  = anchors.find_all('a', href = True)
        for link in links:
            urls = main_url + link.get('href')
            
            #print(urls)
            all_links.append(urls)
    return all_links
path = get_all_url(hyper)

In [9]:
path2 = []
 
for i in path:
    if i not in path2:
        path2.append(i)

In [ ]:
def get_details(urls):
    property_list = []
    for link in urls:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, 'html.parser')
        dom = etree.HTML(str(soup))
        
        name = soup.find('h1', class_ = 'heading__inner').text.strip().split("-")[0]
            #location = soup.find(lambda t: t.name == 'dt' and 'District' in t.text).findNext('dd').string
        property_status = soup.find('div', class_ = 'content').text.split(' ')[0]
        price= soup.find('h1', class_ = 'heading__inner').text.strip().split("-")[1].strip()
            
        bathroom = soup.find(lambda t: t.name == 'dt' and 'Baths' in t.text).findNext('dd').string
        rooms = soup.find('div', class_ = 'property-info__inner container').find_all('li', class_ = 'amenities-items__item')[2].text.split(':')[1]
            except AttributeError:
                bathroom = None
                rooms = None
        except IndexError:
                pass
            
        prop = {
            'name': name,
            'location': location,
            'property': property_status,
            'price': price,
            'bedrooms': rooms,
            'bathrooms' : bathroom,
            'Pool':None,
            'link': link
        }
        property_list.append(prop)
        print('Saving:', prop['name'])
        
    df = pd.DataFrame(property_list)
    print(df.head(20))
    df.to_csv('homecity.csv', header=True, index=False)